In [154]:
import pandas as pd 
import ast

df = pd.read_csv("../datas/movies_metadata.csv")
df.genres = df.genres.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df.genres = df.genres.apply(lambda x: [genre['name'] for genre in x])
df = df.astype({'title':'string'})

df = df.dropna(subset=['title','overview'])

df.head()


C:\Users\feldm\AppData\Local\Temp\ipykernel_1704\1945717855.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datas/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [153]:
# id links
links = pd.read_csv("../datas/links.csv")
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [72]:
df2 = df[["title", "overview", "genres",'id']]
df['id'] = df['id'].astype('int64')


In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(ngram_range=(1,2))

tfid = vectorize.fit_transform(df['title'])

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

def search(title):
    query_vec = vectorize.transform([title])
    similarity = cosine_similarity(query_vec, tfid).flatten()
    indices = np.argpartition(similarity,-5)[-5:]
    results = df.iloc[indices][::-1]
    return results[['title', 'overview', 'release_date','genres','id']]


In [149]:
search('Toy story')

,title,overview,release_date,genres,id
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,"[Animation, Comedy, Family]",862
2997,Toy Story 2,"Andy heads off to Cowboy Camp, leaving his toy...",1999-10-30,"[Animation, Comedy, Family]",863
15348,Toy Story 3,"Woody, Buzz, and the rest of Andy's toys haven...",2010-06-16,"[Animation, Family, Comedy]",10193
21928,Toy Story of Terror!,What starts out as a fun road trip for the Toy...,2013-10-15,"[Animation, Comedy, Family]",213121
25800,Toy Story That Time Forgot,"During a post-Christmas play date, the gang fi...",2014-12-02,"[Animation, Family]",256835


In [88]:
ratings = pd.read_csv('../datas/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [152]:
movie_id = 862
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings['rating'] > 4 )]['userId'].unique()
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings['rating'] > 4)]["movieId"]
similar_user_recs = similar_user_recs.value_counts() /len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .1]
all_user = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
all_users_recs = all_user["movieId"].value_counts() / len(all_user["userId"].unique())
rec_percentages = pd.concat([similar_user_recs,all_users_recs], axis=1)
rec_percentages.columns = ['similar', 'all']
rec_percentages["score"] = rec_percentages['similar'] / rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending= False)

rec_percentages

,similar,all,score
movieId,,,
862,1.000000,0.000386,2590.392405
860,0.202532,0.000391,518.078481
630,0.113924,0.000230,496.032588
1002,0.101266,0.000205,493.408077
1352,0.139241,0.000362,385.058330
...,...,...,...
7153,0.101266,0.132608,0.763649
110,0.101266,0.135007,0.750077
4993,0.101266,0.142958,0.708362


In [151]:
results_reco = rec_percentages.head(10).merge(df, left_index=True, right_on='id')
results_reco[['title','overview','release_date','id']]

,title,overview,release_date,id
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,862
6827,WarGames,High School student David Lightman (Matthew Br...,1983-06-03,860
892,The Wizard of Oz,Young Dorothy finds herself in a magical world...,1939-08-15,630
9211,2046,2046 is the sequel to Wong Kar-Wais’ successfu...,2004-05-20,844
1188,Stalker,"Near a gray and unnamed city is the Zone, a pl...",1979-04-16,1398
3368,Hook,The boy who wasn't supposed to grow up—Peter P...,1991-12-11,879
